In [ ]:
import json
import requests
from flask import Flask
from flask import jsonify, request, abort,Response
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import func
import random
from datetime import datetime,timedelta, date

In [ ]:
app = Flask(__name__)
db = SQLAlchemy(app)
env = os.environ['ENV']
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = "mysql+pymysql://root:abcd1234@db:3306/root"

In [ ]:
db.init_app(app)

In [ ]:
#DB表格(ORM)

class Login(db.Model):
    __tablename__ = "login"
    loginID =               db.Column(db.Integer, primary_key=True)
    phone =                 db.Column(db.String(256), nullable=False,unique=True)
    email =                 db.Column(db.String(512), nullable=False,default = "")
    password =              db.Column(db.String(512), nullable=False)
    createDate =            db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    modifyDate =            db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    isAdmin =               db.Column(db.Boolean , nullable=False , default=0)
    lastLoginDate =         db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    inviteBy =              db.Column(db.Integer, nullable=True)
    buyRate =               db.Column(db.Numeric(20, 10), nullable=False,default=0.026)
    validationFile =        db.Column(db.String(512), nullable=False,default = "")

    def __init__(self
                ,phone
                ,email
                ,password
                ,buyRate=.026
                ,validationFile=""):
        self.phone            = phone
        self.email            = email
        self.password         = password
        self.buyRate          = buyRate
        self.validationFile   = validationFile
        
        
class PhoneValidate(db.Model):
    __tablename__ = "phoneValidate"
    phoneValidateID =    db.Column(db.Integer, primary_key=True)
    phone = db.Column(db.String(20), nullable=False)
    phoneValidateToken = db.Column(db.String(10), nullable=False)
    createDate = db.Column(db.DateTime, nullable=True, default=datetime.utcnow) #albert加入的
    isValidated = db.Column(db.Boolean , nullable=False , default=0)

    def __init__(self, phone, phoneValidateToken):
        self.phone = phone
        self.phoneValidateToken = phoneValidateToken

In [ ]:
def phone_validate():
    result = {
        "message":""
    }
    statusCode = 400
    data = request.get_json()
    bodyKey_list = ['phone']
    for k in bodyKey_list:
        if k not in data:
            result['message'] = "miss body json key"
            return Response(json.dumps(result),status=statusCode,  mimetype='application/json')
    if request.method == 'POST':
        if data is not None:
            query_data = Login.query.filter_by(
                phone = data['phone']
            ).first()
            chinaCode = '+86 '
            if query_data is not None:
                statusCode = 201
                result['message'] = "phone is exist in database."

            else:
                intList = list(range(10))
                random.shuffle(intList) #把10個數字隨機打亂
                validateNoList = intList[:5] #抽出5前個碼當作驗證碼
                validateNoStr = ''.join(map(str, validateNoList)) #透過map函數把str()方法套用到list中每個元素，再以.join把字串連接!
                phoneValidate = PhoneValidate(data['phone'], validateNoStr) #寫入資料庫表格
                db.session.add(phoneValidate)
                db.session.commit() #提交資料表資料
                
                #安全性:防止惡意多次寄送簡訊
                #計算同一支電話號碼簡訊要求次數:和資料庫要資料來查看統計當天日期請求總次數
                #sqlalchemy的func.DATE(DateTime格式)等同MySQL的DATE(datetime格式)，可以抽取出日期'yyyy-mm-dd'
                #python datetime的
                query_data_number = PhoneValidate.query.filter_by(phone = data['phone']).filter(func.DATE(PhoneValidate.createDate) == date.today().isoformat()).count()

                if query_data_number > 5:
                    result['message'] = f"超過當天請求次數5次，總次數:{query_data_number}次"
                    return Response(json.dumps(result),status=statusCode,  mimetype='application/json')

                '''
                可以網上搜尋sns_wrapper
                sns_wrapper = SnsWrapper(boto3.resource('sns', region_name="", aws_access_key_id='', aws_secret_access_key=''))
                sns_wrapper.publish_text_message(phone, validateMessage)
                '''



                sns = boto3.client('sns', region_name="", aws_access_key_id='', aws_secret_access_key='')
                validateMessage = F"您的簡訊驗證碼：{validateNoStr}"
                phone = data['phone'].replace(' ','')
                sns.publish(
                    PhoneNumber = phone,
                    Message = validateMessage
                )

                statusCode = 200
                result['message'] = "ok"
          
                
        else:
            result['message'] = "No body"
        
        return Response(json.dumps(result),status=statusCode,  mimetype='application/json')